## Hyper parameter tuning by SageMaker

1. Prepare Requirements
2. 

**Reference**

* [Hyperparameter Tuning using SageMaker PyTorch Container](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/pytorch_mnist/hpo_pytorch_mnist.ipynb)

## Os level setting

Install packages.

```
sudo yum install gcc72-c++.x86_64
sudo yum install clang
```

g++: Install & link same version of gcc.

## Prepare Requirements

In [1]:
!git pull origin master
!pip install -r ../requirements.txt --ignore-installed

From https://github.com/icoxfog417/allennlp-sagemaker-tuning
 * branch            master     -> FETCH_HEAD
Already up-to-date.
  Using cached https://files.pythonhosted.org/packages/a4/c8/10342a6068a8d156a5947e03c95525d559e71ad62de0f2585ab922e14533/allennlp-0.8.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6f/03/6d7978bc8a43788846d76bff48328cf67132292ab242faac31a7cefa2639/awscli-1.16.135-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e9/69/f5e05f578585ed9935247be3788b374f90701296a70c8871bcd6d21edb00/matplotlib-3.0.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/77/67/2b1fe72bdd13ee9ec32b97959d7dfbfcd7c0548081d69aaf8493c1e695f9/editdistance-0.5.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/5d/3c/d5fa084dd3a82ffc645aba78c417e6072ff48552e3301b1fa3bd711e03d4/pytorch_pretrained_bert-0.6.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org

  Using cached https://files.pythonhosted.org/packages/36/fa/08e9e6e0e3cbd1d362c3bbee8d01d0aedb2155c4ac112b19ef3cae8eed8d/docutils-0.14-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1a/e6/8f50d5442cef835bc83a0555d23305b9210f20eac62990d3ec3911b591d9/botocore-1.12.125-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/69/a7/88719d132b18300b4369fbffa741841cfd36d1e637e1990f27929945b538/kiwisolver-1.0.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/41/17/c62faccbfbd163c7f57f3844689e3a78bae1f403648a6afb1d0866d87fbb/python_dateutil-2.8.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/de/0a/001be530836743d8be6c2d85069f46fecf84ac6c18c7f5fb8125ee11d854/pyparsing-2.3.1-py2.py3-none-any.whl
  Using cached https://files.pyth

  Using cached https://files.pythonhosted.org/packages/bf/45/142141aa47e01a5779f0fa5a53b81f8379ce8f2b1cd13df7d2f1d751ae42/greenlet-0.4.15-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e6/35/f187bdf23be87092bd0f1200d43d23076cee4d0dec109f195173fd3ebc79/mock-2.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/48/68/c3afca1a5aa8d2997ec3b8ee822a4d752cf85907b321f07ea86888545152/docker-3.7.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5b/12/b0409a94dad366d98a8eee2a77678c7a73aafd8c0e4b835abea634ea3896/cryptography-2.6.1-cp34-abi3-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/bf/5c/5fa238c0c5b0656994b52721dd8b1d7bf52ebd8786518dde794f44de86b6/python_jose-2.0.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a4/a5/da7887285564f9e0ae5cd25a453cca36e2cd43d8ccc9effde260b4d80904/aws_xray_sdk-0.95-py2.py3-none-any.whl
  Using cached https://f

  Using cached https://files.pythonhosted.org/packages/20/f7/87b62a8895bf7c93e907b05b97bc4459c81a38a61151f03a6eae13d863aa/cffi-1.12.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ea/cd/35485615f45f30a510576f1a56d1e0a7ad7bd8ab5ed7cdc600ef7cd06222/asn1crypto-0.24.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/f4/73669d51825516ce8c43b816c0a6b64cd6eb71d08b99820c00792cb42222/ecdsa-0.13-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4e/b4/7d007370568d98822c833e0d0e804417620865710dc8a5830bbed58328d1/pycryptodome-3.8.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/dc/12/8c44eabb501e2bc0aec0dd152b328074d98a50968d3a02be28f6037f0c6a/jsonpickle-1.1-py2.py3-none-any.whl
sagemaker 1.18.6.post0 has requirement requests<2.21,>=2.20.0, but you'll have requests 2.21.0 which is incompatible.
docker-compose 1.23.2 has requirement requests!=2.11.0,!=2.12.2,!=2.

In [4]:
!python ../example/train.py

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
93B [00:00, 89383.66B/s]                                                        
93B [00:00, 99229.27B/s]                                                        
2it [00:00, 10880.17it/s]
2it [00:00, 14364.05it/s]
100%|██████████████████████████████████████████| 4/4 [00:00<00:00, 59493.67it/s]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    encoding='utf-8')
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exi

unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    encoding='utf-8')
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
accuracy: 0.5556, loss: 1.0743 ||: 100%|█████████| 1/1 [00:00<00:00, 193.17it/s]
accuracy: 0.4444, loss: 1.0744 ||: 100%|█████████| 1/1 [00:00<00:00, 455.61it/s]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    e

unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    encoding='utf-8')
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
accuracy: 0.4444, loss: 1.0648 ||: 100%|█████████| 1/1 [00:00<00:00, 198.73it/s]
accuracy: 0.4444, loss: 1.0656 ||: 100%|█████████| 1/1 [00:00<00:00, 447.11it/s]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    e

unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    encoding='utf-8')
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
accuracy: 0.4444, loss: 1.0606 ||: 100%|█████████| 1/1 [00:00<00:00, 189.50it/s]
accuracy: 0.4444, loss: 1.0616 ||: 100%|█████████| 1/1 [00:00<00:00, 409.84it/s]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    e

accuracy: 0.4444, loss: 1.0594 ||: 100%|█████████| 1/1 [00:00<00:00, 201.93it/s]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    encoding='utf-8')
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
accuracy: 0.4444, loss: 1.0581 ||: 100%|█████████| 1/1 [00:00<00:00, 191.09it/s]
accuracy: 0.4444, loss: 1.0592 ||: 100%|█████████| 1/1 [00:00<00:00, 469.84it/s]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/p

...

unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    encoding='utf-8')
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
accuracy: 1.0000, loss: 0.0205 ||: 100%|█████████| 1/1 [00:00<00:00, 199.11it/s]
accuracy: 1.0000, loss: 0.0208 ||: 100%|█████████| 1/1 [00:00<00:00, 473.24it/s]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    e

unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    encoding='utf-8')
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
accuracy: 1.0000, loss: 0.0199 ||: 100%|█████████| 1/1 [00:00<00:00, 189.42it/s]
accuracy: 1.0000, loss: 0.0202 ||: 100%|█████████| 1/1 [00:00<00:00, 422.34it/s]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    e

accuracy: 1.0000, loss: 0.0197 ||: 100%|█████████| 1/1 [00:00<00:00, 439.15it/s]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    encoding='utf-8')
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
accuracy: 1.0000, loss: 0.0193 ||: 100%|█████████| 1/1 [00:00<00:00, 196.34it/s]
accuracy: 1.0000, loss: 0.0196 ||: 100%|█████████| 1/1 [00:00<00:00, 471.54it/s]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/p

unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    encoding='utf-8')
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
accuracy: 1.0000, loss: 0.0188 ||: 100%|█████████| 1/1 [00:00<00:00, 207.60it/s]
accuracy: 1.0000, loss: 0.0191 ||: 100%|█████████| 1/1 [00:00<00:00, 433.30it/s]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    e

unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    encoding='utf-8')
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
accuracy: 1.0000, loss: 0.0183 ||: 100%|█████████| 1/1 [00:00<00:00, 182.73it/s]
accuracy: 1.0000, loss: 0.0186 ||: 100%|█████████| 1/1 [00:00<00:00, 475.87it/s]
Spacy models 'en_core_web_sm' not found.  Downloading and installing.
    100% |████████████████████████████████| 11.1MB 79.8MB/s ta 0:00:01
  Running setup.py install for en-core-web-sm ... done
You are using p